# Initialization

In [1]:
import os
from pathlib import Path
import signal
import time

# PyBoolNet initialization
from pyboolnet.repository import get_all_names, get_bnet
from pyboolnet.file_exchange import bnet2primes
from pyboolnet.trap_spaces import compute_trap_spaces

# BioLQM init
import biolqm

# trappist init
import trappist

# timeout with signals
class TimeOutException(Exception):
   pass

def handler(_signnum, _frame):
    print("Timeout")
    raise TimeOutException()
    
signal.signal(signal.SIGALRM, handler)

This notebook has been executed using the docker image `colomoto/colomoto-docker:2022-05-01`

<Handlers.SIG_DFL: 0>

In [2]:
# compute trapspaces using PyBoolNet directly
def bench_pyboolnet(models, max=1000, timeout=120):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            primes = bnet2primes(model + ".bnet")
            end = time.perf_counter()
            print(f"implicants: {end - start}s")
            tspaces = compute_trap_spaces(primes, "min", max_output=max)
            end = time.perf_counter()
            print(f"{len(tspaces)} trap spaces {end - start}s\n===")
        except TimeOutException as ex:
            print("DNF\n===")
        signal.alarm(0)

In [3]:
# compute trapspaces using trappist
def bench_trappist(models, max=1000, timeout=120):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            biolqm.save(biolqm.load(model + ".bnet"), model + ".pnml")
            end = time.perf_counter()
            print(f"conversion: {end - start}s")
            tspaces = trappist.compute_trap_spaces(model + ".pnml", max_output=max)
            end = time.perf_counter()
            print(f"{len(list(tspaces))} trap spaces {end - start}s\n===")
        except TimeOutException as ex:
            print("DNF\n===")
        signal.alarm(0)

## PyBoolNet repository

Unfortunately all models are very easy

In [4]:
# all models from the PyBoolNet repository https://github.com/hklarner/pyboolnet/tree/master/pyboolnet/repository
models = get_all_names()

# only the slowest ones
# models = ['jaoude_thdiff', 'n12c5', 'remy_tumorigenesis', 'selvaggio_emt']

# save .bnet files locally
for model in models:
    net = get_bnet(model)
    with open(model + ".bnet", "wt") as f:
        print(net, file=f)

In [5]:
bench_pyboolnet(models)

jaoude_thdiff
implicants: 1.2828891660319641s
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 1.4223144280258566s
===
n12c5
implicants: 31.515609591966495s
5 trap spaces 31.54398177296389s
===
remy_tumorigenesis
implicants: 1.6913005700334907s
25 trap spaces 1.7177036459906958s
===
selvaggio_emt
implicants: 0.8169596969964914s
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 0.9372284739511088s
===


In [6]:
bench_trappist(models)

jaoude_thdiff
conversion: 0.3728707139962353s
1000 trap spaces 0.5578273219871335s
===
n12c5
conversion: 0.01761878904653713s
5 trap spaces 0.04292340204119682s
===
remy_tumorigenesis
conversion: 0.026393020001705736s
25 trap spaces 0.06600769102806225s
===
selvaggio_emt
conversion: 0.03657483001006767s
1000 trap spaces 0.21081806503934786s
===


## Local set of big models

Collected from various bibliographic sources (see article for details)

In [7]:
# all models from the trappist's benchmark directory
models = [str(Path().absolute() / "models" / name[:-5]) for name in os.listdir("models") if name.endswith(".bnet")]

In [8]:
bench_pyboolnet(models)

/notebook/selected raw models/CASCADE3
implicants: 79.66887543298071s
1 trap spaces 79.71478978800587s
===
/notebook/selected raw models/KYNURENINE-PATHWAY
implicants: 81.425493243034s
1 trap spaces 81.46424335002666s
===
/notebook/selected raw models/SN5
Timeout
DNF
/notebook/selected raw models/ERBB-RECEPTOR-SIGNALING
Timeout
DNF
/notebook/selected raw models/Executable_file_for_cholocystokinin_model
implicants: 0.34361790399998426s
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 1.0530116260051727s
===
/notebook/selected raw models/korkut_2015a
Timeout
DNF
/notebook/selected raw models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
implicants: 0.17193889198824763s
1 trap spaces 0.2433038970339112s
===
/notebook/selected raw models/Corral_ThIL17diff_15jan2021
Timeout
DNF
/notebook/selected raw models/HIV-1
Timeout
DNF
/notebook/selected raw models/Colitis_associated_colon_cancer
implicants: 0.18831702001625672s
5 trap spaces 0.38342

In [9]:
bench_trappist(models)

/notebook/selected raw models/CASCADE3
conversion: 2.969967622018885s
1 trap spaces 9.68063587602228s
===
/notebook/selected raw models/KYNURENINE-PATHWAY


AssertionError: Error while loading model